In [1]:
import pandas as pd
import numpy as np
import matlab.engine
import os
import sys
from tqdm import tqdm

In [2]:
def extract_fit_info(mrs_fit):
    gaba = mrs_fit["GABA"]

    gaba_linewidth = float(gaba["FWHM"])
    gaba_snr = float(gaba["SNR"])
    gaba_fit_error = float(gaba["FitError"])
    gaba_conc_water = float(gaba["ConcIU"])
    gaba_conc_cr = float(gaba["ConcCr"])

    glx = mrs_fit["Glx"]

    glx_linewidth = float(glx["FWHM"])
    glx_snr = float(glx["SNR"])
    glx_fit_error = float(glx["FitError"])
    glx_conc_water = float(glx["ConcIU"])
    glx_conc_cr = float(glx["ConcCr"])

    return {
        "gaba_snr":gaba_snr,
        "gaba_linewidth":gaba_linewidth,
        "gaba_fit_error":gaba_fit_error,
        "gaba_conc_water":gaba_conc_water,
        "gaba_conc_cr":gaba_conc_cr,
        "glx_snr":glx_snr,
        "glx_linewidth":glx_linewidth,
        "glx_fit_error":glx_fit_error,
        "glx_conc_water":glx_conc_water,
        "glx_conc_cr":glx_conc_cr,
    }

In [3]:
eng = matlab.engine.start_matlab()

In [26]:
idx=50
filenames[idx:idx+20]

['g4_s03_range_101_180.mat',
 'g4_s03_range_111_190.mat',
 'g4_s03_range_11_90.mat',
 'g4_s03_range_121_200.mat',
 'g4_s03_range_131_210.mat',
 'g4_s03_range_141_220.mat',
 'g4_s03_range_151_230.mat',
 'g4_s03_range_161_240.mat',
 'g4_s03_range_171_250.mat',
 'g4_s03_range_181_260.mat',
 'g4_s03_range_191_270.mat',
 'g4_s03_range_1_80.mat',
 'g4_s03_range_201_280.mat',
 'g4_s03_range_211_290.mat',
 'g4_s03_range_21_100.mat',
 'g4_s03_range_221_300.mat',
 'g4_s03_range_231_310.mat',
 'g4_s03_range_241_320.mat',
 'g4_s03_range_31_110.mat',
 'g4_s03_range_41_120.mat']

In [4]:
read_folder = "data/gannet_acc_fit_ranges/"
filenames = os.listdir(read_folder)

pre_df = []
for filename in filenames:
    
    range_start = filename.split(".")[0].split("_")[-2]
    range_end = filename.split(".")[0].split("_")[-1].split(".")[0]
    file_id = filename[:6]
    if file_id[:3]=="p10":
        file_id=filename[:7]
    #
    mrs_fit = eng.load(f"{read_folder}/{filename}")["mrs_fit"]["vox1"]
    pre_row = {"filename":file_id,"range":f"{range_start}-{range_end}"}
    mrs_fit_info = extract_fit_info(mrs_fit)
    pre_row.update(mrs_fit_info)
    pre_df.append(pre_row)

df = pd.DataFrame(pre_df)

In [5]:
df.to_csv("eval/fit_ranges/acc_ref_df.csv",index=False)

In [48]:
df.to_csv("window_sliding_csvs/acc.csv",index=False)

In [16]:
df.shape[0]/1

(3600, 12)

In [50]:
df.groupby("filename").agg({"gaba_conc_water":"std","gaba_conc_cr":"std"}).median()

gaba_conc_water    0.178824
gaba_conc_cr       0.010159
dtype: float64